In [22]:
import pandas as pd
import numpy as np
import time
import subprocess
import gc

In [23]:
def base_process(readpath):   #Unix时间脱敏，并转化为中国时区，平移24小时，则train数据集的时间为1日~8日上午，test数据集的时间为8日下午
    df = pd.read_csv(readpath,sep=" ")
    df["context_timestamp"] = df["context_timestamp"].apply(lambda x:time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(x)))
    df["context_timestamp"] = pd.DatetimeIndex(df["context_timestamp"]).shift(+24,freq="H")     
    df["context_timestamp"] = pd.to_datetime(df["context_timestamp"],unit="s")
    df["day"] = df["context_timestamp"].dt.day
    df["hour"] = df["context_timestamp"].dt.hour
    return df

In [3]:
%%time
train = base_process("../data/raw_data/round2_ijcai_18_train_20180425.txt")

Wall time: 22min 32s


In [24]:
%%time
test = base_process("../data/raw_data/round2_ijcai_18_test_b_20180510.txt")

Wall time: 1min 22s


In [26]:
from memoryOptimize import memory_optimze

In [5]:
train = memory_optimze(train)

**********正在统计数据所占内存大小****************
*******处理int型***************
******处理float型**************
**********处理object型（字符型）**************
优化前，数据集所占内存大小为11680.738045692444M
优化后，数据集所占内存大小为1553.6165294647217M
优化后，内存减小了86.6993290716107%


In [8]:
train.to_pickle("../data/raw_data/base_preprocess_train.pickle") #生成pickle文件，有利于减少内存

In [27]:
test = memory_optimze(test)
test.to_pickle("../data/raw_data/base_preprocess_test.pickle")

**********正在统计数据所占内存大小****************
*******处理int型***************
******处理float型**************
**********处理object型（字符型）**************
优化前，数据集所占内存大小为1349.7871475219727M
优化后，数据集所占内存大小为225.00482368469238M
优化后，内存减小了83.33034774425205%


In [2]:
train = pd.read_pickle("../data/raw_data/base_preprocess_train.pickle")
X_tr = train.drop(["is_trade","item_category_list","item_property_list","predict_category_property","context_timestamp"],axis=1)#同时丢弃类目特征及时间特征
y_tr = train["is_trade"]

In [ ]:
test = pd.read_pickle("../data/raw_data/base_preprocess_test.pickle")
X_ts = test.drop(["item_category_list","item_property_list","predict_category_property","context_timestamp"],axis=1) #同时丢弃类目特征及时间特征

In [30]:
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr, test_size=0.3, random_state=0)

In [6]:
X_train = X_train.drop(["item_category_list","item_property_list","predict_category_property","context_timestamp"],axis=1)

In [8]:
X_test = X_test.drop(["item_category_list","item_property_list","predict_category_property","context_timestamp"],axis=1)

In [31]:
clf = XGBClassifier(
    n_estimators=100,#三十棵树
    learning_rate =0.3,
    max_depth=3,
    min_child_weight=1,
    gamma=0.3,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=12,
    scale_pos_weight=1,
    reg_lambda=1,
    seed=27)

In [32]:
model_sklearn=clf.fit(X_tr, y_tr)

In [40]:
gc.collect()

80

In [34]:
y_sklearn= clf.predict_proba(X_ts)[:,1]

In [35]:
from sklearn import metrics

In [37]:
#获取每个样本在每颗树叶子节点的索引值
test_new_feature= clf.apply(X_ts)

In [1]:
train_new_feature= clf.apply(X_tr)

In [16]:
train_new_feature2 = pd.DataFrame(train_new_feature)

In [17]:
test_new_feature2 = pd.DataFrame(test_new_feature)

In [18]:
print("新的特征集(sklearn接口)：",train_new_feature2.shape)
print("新的测试集(sklearn接口)：",test_new_feature2.shape)  

新的特征集(sklearn接口)： (7302425, 100)
新的测试集(sklearn接口)： (3129611, 100)


In [19]:
test_new_feature2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,13,10,8,10,8,8,7,9,10,13,...,13,14,14,13,9,10,8,10,10,7
1,13,10,8,10,8,8,8,8,9,13,...,14,13,7,12,8,10,14,8,10,13
2,12,9,7,9,7,7,7,7,7,12,...,13,13,7,13,10,10,11,7,9,11
3,13,10,8,10,8,8,7,9,9,13,...,13,14,14,14,9,10,14,11,10,7
4,13,10,8,10,8,8,7,7,10,13,...,13,14,7,14,10,10,8,8,9,11
5,12,13,7,13,7,9,9,7,11,12,...,13,14,7,13,14,10,14,7,10,13
6,14,10,8,10,8,8,8,9,9,14,...,8,9,7,8,9,10,14,8,7,7
7,13,10,8,10,8,8,7,7,9,13,...,13,14,7,14,10,10,7,8,14,11
8,14,10,8,10,8,8,7,9,9,14,...,7,14,14,13,9,11,14,14,7,7
9,9,8,8,8,8,8,7,7,7,7,...,13,14,7,11,8,9,8,7,14,13


In [8]:
%%time
train = base_process("../data/raw_data/round2_ijcai_18_train_b_20180510.txt")
test = base_process("../data/raw_data/round2_ijcai_18_test_b_20180510.txt")

Wall time: 1min 1s


In [9]:
train.shape
test.shape

(1209768, 28)

In [13]:
%%time
train_x = train.drop(["is_trade"],axis=1)
df = train_x.append(test,ignore_index=True)  #合并训练集与测试集

Wall time: 16min 39s


In [14]:
adClick = df.groupby(["user_id"])["context_timestamp"].count() #统计每位用户总点击量

In [15]:
dayClick = df.pivot_table(index="user_id",columns="day",values="context_timestamp",aggfunc="count") #统计每位用户在每一天的点击量

In [16]:
dayClick = dayClick.fillna(0)  #填充0
dayClick_currence = dayClick.iloc[:,-1] #最近一次点击量

In [17]:
dayClick_mean = dayClick.mean(axis=1) #日均点击量
dayClick_max = dayClick.max(axis=1) #日最大点击量
dayClick_min = dayClick.min(axis=1) #日最小点击量
dayClick_median = dayClick.median(axis=1)#日中位数点击量

In [18]:
%%time
df = df.sort_values(by="context_timestamp") #按照时间排序

Wall time: 2h 28min 12s


In [19]:
df["lastClick"] = df.groupby(["user_id"])[["context_timestamp"]].shift(1) #距离点击上一次的时间距离
df["clickDist"] = (df["context_timestamp"] - df["lastClick"])
df.iloc[:,-1] = df.iloc[:,-1].map(lambda x:x.total_seconds())

In [20]:
%%time
dist_max = df.groupby("user_id")[["clickDist"]].max() #点击时间间隔
dist_min = df.groupby("user_id")[["clickDist"]].min()

Wall time: 1min 24s


In [22]:
%%time
dist_mean = df.groupby("user_id")[["clickDist"]].mean()
dist_median = df.groupby("user_id")[["clickDist"]].median()

Wall time: 7.71 s


In [23]:
%%time
dist_std = df.groupby("user_id")["clickDist"].agg(np.std)
dist_std = dist_std.reset_index()
dist_std.columns = ["user_id","std"]
dist_std = dist_std.set_index("user_id")

Wall time: 3.29 s


In [24]:
var_ls = [adClick,dayClick_currence,dayClick_mean,dayClick_max,dayClick_min,dayClick_median,dist_max,dist_max,dist_mean,dist_median,dist_std]
var_name_ls = ["adClick","dayClick_currence",
                "dayClick_mean","dayClick_max","dayClick_min","dayClick_median",
                "dist_max","dist_min","dist_mean","dist_median","dist_std"]
clickData = df[["user_id"]].drop_duplicates() #对用户ID去重

In [25]:
for var,name in zip(var_ls,var_name_ls):
    var = var.reset_index()
    var.columns = ["user_id",name]
    clickData = pd.merge(clickData,var,how="left",left_on="user_id",right_on="user_id")

clickData = clickData.fillna(0) #填充0
clickData_path = "../data/temp_data/_2_get_click_stats.pickle"
clickData.to_pickle(clickData_path)
print("保存在.... ",clickData_path)

保存在....  ../data/temp_data/_2_get_click_stats.pickle


In [26]:
%%time
temp = df[["user_id","context_timestamp"]].drop_duplicates()
earlist = temp.groupby("user_id")['context_timestamp'].min().reset_index()  #最早日期
latest = temp.groupby("user_id")['context_timestamp'].max().reset_index()    #最迟日期
earlist = earlist.rename(columns={"context_timestamp":"earlist_date"})  #重命名
latest = latest.rename(columns={"context_timestamp":"latest_date"})#重命名
temp = pd.merge(temp,earlist,how="left") 
temp = pd.merge(temp,latest,how="left")
temp["first_2_last_dist"] = (temp["latest_date"] - temp["earlist_date"]) #计算第一次和最后一次点击差

Wall time: 28.5 s


In [27]:
#计算当前点击时间与第一次点击时间、最后一次点击时间的时间差
temp["first_2_last_dist"] = temp["first_2_last_dist"].map(lambda x:x.total_seconds())
temp["now_2_first_dist"] = (temp["context_timestamp"] - temp["earlist_date"])
temp["now_2_first_dist"] = temp["now_2_first_dist"].map(lambda x:x.total_seconds()) 
temp["now_2_last_dist"] = (temp["latest_date"] - temp["context_timestamp"])
temp["now_2_last_dist"] = temp["now_2_last_dist"].map(lambda x:x.total_seconds()) 

In [28]:
%%time
temp = temp.fillna(0)
temp = temp[["user_id","context_timestamp","first_2_last_dist","now_2_first_dist","now_2_last_dist"]]
path = "../data/temp_data/_2_first_time_to_last_margin.pickle"
temp.to_pickle(path)

Wall time: 10.2 s


In [29]:
def get_last_2(x):
    try:
        a = x.iloc[-2]
    except:
        a = np.nan
    return a

def get_last_3(x):
    try:
        a = x.iloc[-3]
    except:
        a = np.nan
    return a

def get_last_4(x):
    try:
        a = x.iloc[-4]
    except:
        a = np.nan
    return a

temp = df[["user_id","context_timestamp"]].drop_duplicates() 
#temp = temp.sort_values(by=["context_timestamp"])
lastClick_time = temp.groupby(["user_id"])["context_timestamp"].max() #最后一次点击
last2Click_time = temp.groupby(["user_id"])["context_timestamp"].agg(get_last_2) #倒数第二次点击
last3Click_time = temp.groupby(["user_id"])["context_timestamp"].agg(get_last_3) #倒数第三次点击

var_name_ls = ["lastClick_time","last2Click_time","last3Click_time"]
var_ls = [lastClick_time,last2Click_time,last3Click_time]

In [30]:
#合并
for var,name in zip(var_ls,var_name_ls):
    var = var.reset_index()
    var.columns = ["user_id",name]
    temp = pd.merge(temp,var,how="left")

        #最后一次与倒数第二次
temp["last_click_1_2_dist"] = temp["lastClick_time"] - temp["last2Click_time"]
temp["last_click_1_2_dist"] = temp["last_click_1_2_dist"].map(lambda x:x.total_seconds())

        #倒数第二次与倒数第三次
temp["last_click_2_3_dist"] = temp["last2Click_time"] - temp["last3Click_time"]
temp["last_click_2_3_dist"] = temp["last_click_2_3_dist"].map(lambda x:x.total_seconds())

        #最后一次与倒数第3次
temp["last_click_1_3_dist"] = temp["lastClick_time"] - temp["last3Click_time"]
temp["last_click_1_3_dist"] = temp["last_click_1_3_dist"].map(lambda x:x.total_seconds())

temp = temp.drop(["context_timestamp","lastClick_time","last2Click_time","last3Click_time"],axis=1)
temp = temp.fillna(0)

path = "../data/temp_data/_2_last_1_2_dist.pickle"
print("保存在... ",path)
temp.to_pickle(path)

保存在...  ../data/temp_data/_2_last_1_2_dist.pickle


In [31]:
%%time
history_fea = df[["user_id"]].drop_duplicates() #获取关联匹配的id    #历史统计点击次数
for i in range(7,9): 
    temp = df[df["day"]<i] 
    stats = temp.groupby("user_id",as_index=False)["context_timestamp"].count() #点击次数
    stats.columns = ["user_id","click_cnt"]
    new_columns = [col if col=="user_id" else '{0}_{1}'.format(i,col) for col in stats.columns]
    stats.columns = new_columns#关联
    history_fea = pd.merge(history_fea,stats,how="left",left_on="user_id",right_on="user_id")
history_fea  = history_fea.drop_duplicates().fillna(0) #填充为0
path = "../data/temp_data/_2_history_stats.pickle"
history_fea.to_pickle(path) #保存
print("保存在",path)

保存在 ../data/temp_data/_2_history_stats.pickle
Wall time: 3h 24min 1s


In [ ]:
#########内存溢出#############
ads_feature = ['item_id','item_category_list','item_property_list'
                   ,'item_brand_id','item_city_id',"category_0","category_1","category_2"]
context_feature = ['shop_id','context_page_id'] #上下文
stats_feature = ads_feature+context_feature
    
#统计每一个用户每一天点击每一个feature的次数
for feature in stats_feature:
    temp_path = "../data/temp_data/_2_user_day_click_{0}".format(feature)
    user_feature_click_day = df.groupby(['user_id','day',feature]).size().reset_index().rename(columns={0:'user_'+feature+'_click_day'})
    user_feature_click_day.to_pickle(temp_path)
    print("保存在{0}".format(temp_path))

In [32]:
%%time
last_day=8
Id_feature =["item_id","item_category_list","item_property_list","item_brand_id"
            ,"item_city_id","user_id","user_gender_id"
            ,"user_occupation_id","shop_id"]
    
df = df[df["day"]<=8]
for i in Id_feature:
    path = "../data/temp_data/_3_ID_global_sum_cnt_{0}.pickle".format(i)
    feature_count_sum = pd.DataFrame(df.groupby(i).size()).reset_index().rename(columns={0:i+'_sum_count'})
    feature_count_sum.to_pickle(path)
    del feature_count_sum
    gc.collect()
    print("保存... ",path)

保存...  ../data/temp_data/_3_ID_global_sum_cnt_item_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_item_category_list.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_item_property_list.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_item_brand_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_item_city_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_user_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_user_gender_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_user_occupation_id.pickle
保存...  ../data/temp_data/_3_ID_global_sum_cnt_shop_id.pickle
Wall time: 2h 26min 6s
Compiler : 114 ms


In [4]:
from smooth import HyperParam #平滑函数
import copy

In [2]:
df = copy.deepcopy(train) #copy train

In [5]:
#统计每一个feature每天的转化次数、曝光次数
for feat_1 in ['item_id','item_category_list','item_property_list','item_brand_id','item_city_id'
              ,"user_id","shop_id","context_id"]:
    res = pd.DataFrame()
    Id = train[[feat_1]].drop_duplicates() # feature,用来关联点击总次数、转化次数
    for day in range(1,8):
        maskTemp = df[df["day"]<=day] #选取当天之前的所有天数
        click_cnt = maskTemp.groupby(feat_1,as_index=False)["is_trade"].count()
        click_cnt = click_cnt.rename(columns={"is_trade":"{0}_click_cnt".format(feat_1)}) #统计feat1总点击数
        temp = pd.merge(Id,click_cnt,how="left")
                
        #转化记录
        click_trade_cnt = maskTemp.groupby(feat_1,as_index=False)["is_trade"].sum()
        click_trade_cnt = click_trade_cnt.rename(columns={"is_trade":"{0}_trade_cnt".format(feat_1)}) 
        temp = pd.merge(temp,click_trade_cnt,how="left")
        temp = temp.fillna(0)
        temp["day"] = day
                
        #合并
        res = pd.concat([res,temp],ignore_index=True,axis=0) 
    res.to_pickle("../data/temp_data/_4_{0}_trade_cnt.pickle".format(feat_1))
    print("保存到 ../data/temp_data/_4_{0}_trade_cnt.pickle".format(feat_1))


KeyboardInterrupt



In [6]:
del df
del Id
del res
gc.collect()

77

In [ ]:
for feat_1,feat_2 in[("item_id","user_id"),("item_id","user_gender_id"),("user_id","item_brand_id"),("user_id","shop_id")
                     ("item_id","context_id"),("user_id","item_property_list"),("user_id","item_city_id")]:
    res = pd.DataFrame()
    Id = train[[feat_1,feat_2]].drop_duplicates() # feature,用来关联点击总次数、转化次数
    for day in range(1,8):
        maskTemp = df[df["day"]<=day] #选取当天之前的所有天数
        click_cnt = maskTemp.groupby([feat_1,feat_2],as_index=False)["is_trade"].count()
        click_cnt = click_cnt.rename(columns={"is_trade":"{0}_{1}_click_cnt".format(feat_1,feat_2)}) #统计feat1总点击数
        temp = pd.merge(Id,click_cnt,how="left",left_on=[feat_1,feat_2],right_on=[feat_1,feat_2])
                
        #转化记录
        click_trade_cnt = maskTemp.groupby([feat_1,feat_2],as_index=False)["is_trade"].sum()
        click_trade_cnt = click_trade_cnt.rename(columns={"is_trade":"{0}_{1}_trade_cnt".format(feat_1,feat_2)}) 
        temp = pd.merge(temp,click_trade_cnt,how="left",left_on=[feat_1,feat_2],right_on=[feat_1,feat_2])
        temp = temp.fillna(0)
        temp["day"] = day
                
                #合并
        res = pd.concat([res,temp],ignore_index=True,axis=0) 
    res.to_pickle("../data/temp_data/_4_{0}_and_{1}_trade_cnt.pickle".format(feat_1,feat_2))
    print("保存到 ../data/temp_data/_4_{0}_and_{1}_trade_cnt.pickle".format(feat_1,feat_2))

for feat_1,feat_2,feat_3 in [("item_id","user_id","shop_id"),("item_id","user_id","context_id")]:
    res = pd.DataFrame()
    Id = train[[feat_1,feat_2,feat_3]].drop_duplicates() # feature,用来关联点击总次数、转化次数
    for day in range(18,26):
        maskTemp = df[df["day"]<day] #选取当天之前的所有天数
        click_cnt = maskTemp.groupby([feat_1,feat_2,feat_3],as_index=False)["is_trade"].count()
        click_cnt = click_cnt.rename(columns={"is_trade":"{0}_{1}_{2}_click_cnt".format(feat_1,feat_2,feat_3)}) #统计feat1总点击数
        temp = pd.merge(Id,click_cnt,how="left",left_on=[feat_1,feat_2,feat_3],right_on=[feat_1,feat_2,feat_3])
                
        #转化记录
        click_trade_cnt = maskTemp.groupby([feat_1,feat_2,feat_3],as_index=False)["is_trade"].sum()
        click_trade_cnt = click_trade_cnt.rename(columns={"is_trade":"{0}_{1}_{2}_trade_cnt".format(feat_1,feat_2,feat_3)}) 
        temp = pd.merge(temp,click_trade_cnt,how="left",left_on=[feat_1,feat_2,feat_3],right_on=[feat_1,feat_2,feat_3])
        temp = temp.fillna(0)
        temp["day"] = day
                
         #合并
        res = pd.concat([res,temp],ignore_index=True,axis=0) 
    res.to_pickle("../data/temp_data/_4_{0}_{1}_and_{2}_trade_cnt.pickle".format(feat_1,feat_2,feat_3))
    print("保存到 ../data/temp_data/_4_{0}_{1}_and_{2}_trade_cnt.pickle".format(feat_1,feat_2,feat_3))

In [ ]:
#对点击率相关特征进行贝叶斯平滑，修正实际CTR
smooth_cvr_log = {}
for feat_1 in ['item_id','item_category_list','item_property_list','item_brand_id','item_city_id'
              ,"user_id","shop_id","context_id"]:
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_{1}_{2}.pickle".format(feature,6,8)
    readPath = "../data/temp_data/_4_{0}_trade_cnt.pickle".format(feature) 
    temp = pd.read_pickle(readPath) #读取需要处理的表
    temp = temp[(temp["day"]>=6)&(temp["day"]<=8)]
    print(temp.shape)
    HP = HyperParam(1, 1) 
    #HP.update_from_data_by_moment(temp[feature + '_click_cnt'].values, temp[feature + '_trade_cnt'].values)
    HP.update_from_data_by_FPI(temp[feature + '_click_cnt'].values, temp[feature + '_trade_cnt'].values,1000,0.001)
    temp[feature + '_smooth'] = (temp[feature + '_trade_cnt'] + HP.alpha) / (temp[feature + '_click_cnt'] + HP.alpha + HP.beta)
    temp.to_pickle(savePath)
    smooth_cvr_log[feature + '_smooth'] = HP.alpha/(HP.alpha+HP.beta)
    print("***************************")
    print("{0}对应的alpha为{1}，beta为{2}".format(feature,HP.alpha,HP.beta))
    print("****************************")
    print("保存{0}".format(savePath))
    del temp
    gc.collect()
    
for feat_1,feat_2 in[("item_id","user_id"),("item_id","user_gender_id"),("user_id","item_brand_id"),("user_id","shop_id")
                     ("item_id","context_id"),("user_id","item_property_list"),("user_id","item_city_id")]:
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_and_{1}_{2}_{3}.pickle".format(feat_1,feat_2,6,8)
    readPath = "../data/temp_data/_4_{0}_and_{1}_trade_cnt.pickle".format(feat_1,feat_2) 
    temp = pd.read_pickle(readPath) #读取需要处理的表
    temp = temp[(temp["day"]>=6)&(temp["day"]<=8)]
    print(temp.shape)
    print("平滑特征{0}and{1}".format(feat_1,feat_2))
    HP = HyperParam(1, 1)
    #HP.update_from_data_by_moment(temp[feat_1+"_"+feat_2 + '_click_cnt'].values, temp[feat_1+"_"+feat_2 + '_trade_cnt'].values)
    HP.update_from_data_by_FPI(temp[feat_1+"_"+feat_2 + '_click_cnt'].values, temp[feat_1+"_"+feat_2 + '_trade_cnt'].values,1000,0.001)
    temp[feat_1+"_"+feat_2 + '_smooth'] = (temp[feat_1+"_"+feat_2 + '_trade_cnt'] + HP.alpha) / (temp[feat_1+"_"+feat_2 + '_click_cnt'] + HP.alpha + HP.beta)
    smooth_cvr_log[feat_1+"_"+feat_2 + '_smooth'] = HP.alpha/(HP.alpha + HP.beta) #记录每一特征的初始值
    temp.to_pickle(savePath)
    del temp
    gc.collect()
    print("***************************")
    print("{0}和{1}组合特征对应的alpha为{2}，beta为{3}".format(feat_1,feat_2,HP.alpha,HP.beta))
    print("****************************")
    print("保存{0}".format(savePath))
    
for feat_1,feat_2,feat_3 in [("item_id","user_id","shop_id"),("item_id","user_id","context_id")]:
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_{1}_and_{2}_{3}_{4}.pickle".format(feat_1,feat_2,feat_3,6,8)
    readPath = "../data/temp_data/_4_{0}_{1}_and_{2}_trade_cnt.pickle".format(feat_1,feat_2,feat_3)
    temp = pd.read_pickle(readPath)
    temp = temp[(temp["day"]>=6)&(temp["day"]<=8)]
    print(temp.shape)
    print("平滑特征{0}and{1}and{2}".format(feat_1,feat_2,feat_3))
    HP = HyperParam(1, 1) 
    #HP.update_from_data_by_moment(temp[feat_1+"_"+feat_2 +"_"+feat_3+ '_click_cnt'].values, temp[feat_1+"_"+feat_2 +"_"+feat_3+ '_trade_cnt'].values)
    HP.update_from_data_by_FPI(temp[feat_1+"_"+feat_2 +"_"+feat_3+ '_click_cnt'].values, temp[feat_1+"_"+feat_2 +"_"+feat_3+ '_trade_cnt'].values,1000,0.001)
    temp[feat_1+"_"+feat_2 +"_"+feat_3+'_smooth'] = (temp[feat_1+"_"+feat_2+"_"+feat_3+ '_trade_cnt'] + HP.alpha) / (temp[feat_1+"_"+feat_2 +"_"+feat_3+ '_click_cnt'] + HP.alpha + HP.beta)
    smooth_cvr_log[feat_1+"_"+feat_2 +"_"+feat_3+ '_smooth'] = HP.alpha/(HP.alpha + HP.beta) #记录每一特征的初始值
    temp.to_pickle(savePath)
    del temp  
    gc.collect()
    print("***************************")
    print("{0}和{1}和{2}组合特征对应的alpha为{3}，beta为{4}".format(feat_1,feat_2,feat_3,HP.alpha,HP.beta))
    print("****************************")
    print("保存{0}".format(savePath))
    
print("保存每个特征的经验初始值，用于之后的填充空值")
with open("../data/smooth_cvr/_4_smooth_cvr_log.pickle","wb") as f1:
    pickle.dump(smooth_cvr_log,f1)

In [3]:
#使用train最后两天的数据
train_set = train[(train["day"]>=7)&(train["day"]<=8)] 
train_set.shape

In [10]:
test_set = test

In [12]:
#合并基本点击数统计
click_stats = pd.read_pickle("../data/temp_data/_2_get_click_stats.pickle") #基本点击数

In [13]:
click_stats.shape

(3282505, 12)

In [14]:
train_set = pd.merge(train_set,click_stats,how="left",left_on="user_id",right_on="user_id")

In [15]:
train_set.shape

(3011618, 40)

In [16]:
test_set = pd.merge(test_set,click_stats,how="left",left_on="user_id",right_on="user_id")
del click_stats
gc.collect()

491

In [17]:
#合并点击时间间隔表
time_dist = pd.read_pickle("../data/temp_data/_2_first_time_to_last_margin.pickle")

In [18]:
time_dist = time_dist.drop_duplicates(["user_id","first_2_last_dist"],keep="last")

In [19]:
time_dist.shape

(3282505, 5)

In [20]:
train_set = pd.merge(train_set,time_dist[["user_id","first_2_last_dist"]],how="left",left_on=["user_id"],right_on=["user_id"])

In [21]:
test_set = pd.merge(test_set,time_dist[["user_id","first_2_last_dist"]],how="left",left_on=["user_id"],right_on=["user_id"]) 
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()

In [22]:
test_set.shape

(1209768, 40)

In [23]:
#合并当前点击行为到第一次和最后一次点击的行为间隔
cols = ["user_id","context_timestamp","now_2_first_dist","now_2_last_dist"]
train_set = pd.merge(train_set,time_dist[cols],how="left",on=["user_id","context_timestamp"])
test_set = pd.merge(test_set,time_dist[cols],how="left",on=["user_id","context_timestamp"]) 
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()
del time_dist
gc.collect()

68

In [24]:
#合并倒数第一次点击和倒数第二次点击行为的间隔
last_1_2_dist =  pd.read_pickle("../data/temp_data/_2_last_1_2_dist.pickle")

In [25]:
last_1_2_dist = last_1_2_dist.drop_duplicates(["user_id","last_click_1_2_dist","last_click_1_3_dist","last_click_2_3_dist"],keep="last")

In [26]:
last_1_2_dist.shape

(3282505, 4)

In [27]:
train_set = pd.merge(train_set,last_1_2_dist,how="left",on=["user_id"])
test_set = pd.merge(test_set,last_1_2_dist,how="left",on=["user_id"]) 
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()
del last_1_2_dist
gc.collect()

35

In [28]:
#合并滑动窗口的间隔
history_stats =  pd.read_pickle("../data/temp_data/_2_history_stats.pickle")

In [29]:
history_stats.shape

(3282505, 3)

In [30]:
train_set = pd.merge(train_set,history_stats,how="left",on=["user_id"])
test_set = pd.merge(test_set,history_stats,how="left",on=["user_id"]) 
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()
del history_stats
gc.collect()

35

In [31]:
save_path_train = "../data/merge_data/_5_merge_data_train_basic_click.pickle"
save_path_test = "../data/merge_data/_5_merge_data_test_basic_click.pickle"
print("save the train and test files !!!")
train_set = train_set.fillna(0)
test_set = test_set.fillna(0)

save the train and test files !!!


In [32]:
%%time
train_set.to_pickle(save_path_train)

Wall time: 1min 29s


In [33]:
test_set.to_pickle(save_path_test)
print("save the files on {0}".format(save_path_train))
print("save the files on {0}".format(save_path_test))    

save the files on ../data/merge_data/_5_merge_data_train_basic_click.pickle
save the files on ../data/merge_data/_5_merge_data_test_basic_click.pickle


In [34]:
feature_ls = ["item_id","item_category_list","item_property_list","item_brand_id"
            ,"item_city_id","user_id","user_gender_id"
            ,"user_occupation_id","shop_id"]

In [35]:
cols = ["instance_id"]
cols = feature_ls + cols
train_g = train[cols][train["day"]>=7]
test_g = test[cols]

In [36]:
train_g.shape

(3011618, 10)

In [37]:
for i in feature_ls:
    path = "../data/temp_data/_3_ID_global_sum_cnt_{0}.pickle".format(i)
    temp = pd.read_pickle(path)
    train_g = pd.merge(train_g,temp,how="left",on = [i]).drop_duplicates()
    test_g = pd.merge(test_g,temp,how="left",on = [i]).drop_duplicates()
    del temp
    gc.collect() 

In [38]:
test_g.shape

(1209768, 19)

In [39]:
train_g = train_g.drop(feature_ls,axis=1)
test_g = test_g.drop(feature_ls,axis=1)
        
train_g.to_pickle("../data/merge_data/_5_merge_data_train_ID_global_sum.pickle") #保存
test_g.to_pickle("../data/merge_data/_5_merge_data_test_ID_global_sum.pickle") #保存
print("保存在{0}！！".format("train"))
print("保存在{0}！！".format("test"))
del train_g
del test_g
gc.collect()

保存在train！！
保存在test！！


14

In [40]:
train_set.shape

(3011618, 48)

In [90]:
#########对连续变量进行OneHot############(可以不做)
cols_1 = ["item_price_level","item_sales_level","item_collected_level",
           "user_star_level","item_pv_level","user_gender_id","user_age_level",
           "user_occupation_id","shop_review_num_level","shop_star_level",]
cols_2 = ["shop_review_positive_rate","shop_score_service","shop_score_delivery",
              "shop_score_description"]
id_col = ["instance_id"]
cols = id_col + cols_1 + cols_2 
    
    #-------合并train和test并且打上标签------------
train_set= train_set[cols]
test_set = test_set[cols]
train_set["is_train"] = 1
test_set["is_train"] = 0
data = pd.concat([train_set,test_set],axis=0,ignore_index = True)
#one-hot encoding
data = pd.get_dummies(data,columns=cols_1)
train_set = data[data["is_train"]==1]
test_set = data[data["is_train"]==0]
train_set.to_pickle("../data/merge_data/_5_merge_data_train_originated.pickle") #保存到文件
test_set.to_pickle("../data/merge_data/_5_merge_data_test_originated.pickle")  #保存到文件
print("保存在train")
print("保存在test")

In [ ]:
print("保存每个特征的经验初始值，用于之后的填充空值")
with open("../data/smooth_cvr/_4_smooth_cvr_log.pickle","wb") as f1:
    pickle.dump(smooth_cvr_log,f1)

In [ ]:
#读取每一个特征对应的转换率经验初始值（alpha / (alpha + beta)）
smooth_cvr_log = pd.read_pickle("../data/smooth_cvr/_4_smooth_cvr_log.pickle")
save_path_train = "../data/merge_data/_5_merge_data_train_smooth_cvr_{0}_{1}.pickle".format(6,8)
save_path_test = "../data/merge_data/_5_merge_data_test_smooth_cvr_{0}_{1}.pickle".format(6,8)

In [ ]:
train_s = train[(train["day"]>=6)&(train["day"]<=8)] #
test_s = test[(test["day"]>=6)&(test["day"]<=8)]
feature_1 = ['item_id','item_category_list','item_property_list','item_brand_id'
              ,"item_city_id","user_id","shop_id"]
feature2 = ["item_id","user_id","user_gender_id","item_brand_id","shop_id"
            "context_id","item_property_list","item_city_id"]
feature_3 = ["day"]
combine_feature = [("item_id","user_id"),("item_id","user_gender_id"),("user_id","item_brand_id"),("user_id","shop_id")
                     ("item_id","context_id"),("user_id","item_property_list"),("user_id","item_city_id")]
three_feature = [("item_id","user_id","shop_id"),("item_id","user_id","context_id")]
cols = ["instance_id"]
cols = list(set(cols + feature_1 + feature_2 + feature_3 + context + shop))
train_s = train_s[cols].drop_duplicates()
test_s = test_s[cols].drop_duplicates()

In [ ]:
for i in feature_1:
        #合并每一个特征
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_{1}_{2}.pickle".format(i,timeWindow[0],timeWindow[1])
    temp = pd.read_pickle(savePath).drop_duplicates()
    temp_tr = pd.merge(train[["instance_id",i,"day"]],temp,how="left",on=[i,"day"]).drop_duplicates()
    temp_ts = pd.merge(test[["instance_id",i,"day"]],temp,how="left",on=[i,"day"]).drop_duplicates()
    temp_tr = temp_tr[["instance_id",i + '_smooth']]
    temp_ts = temp_ts[["instance_id",i + '_smooth']]
            
            #填充空值
    temp_tr = temp_tr.fillna(smooth_cvr_log[i + '_smooth'])
    temp_ts = temp_ts.fillna(smooth_cvr_log[i + '_smooth'])
            #保存
    temp_tr.to_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_merge_0.pickle".format(i))
    temp_ts.to_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_merge_0.pickle".format(i))
    print("保存在 ../data/merge_data/_5_smooth_cvr_train_{0}_merge_0.pickle".format(i))
    print("保存在 ../data/merge_data/_5_smooth_cvr_test_{0}_merge_0.pickle".format(i))
    del temp
    gc.collect()
            
        #组合双特征组合的转化率
for feat_1,feat_2 in combine_feature:
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_and_{1}_{2}_{3}.pickle".format(feat_1,feat_2,timeWindow[0],timeWindow[1])
    temp = pd.read_pickle(savePath)
    temp_tr = pd.merge(train[["instance_id","day",feat_1,feat_2]],temp,on=["day",feat_1,feat_2],how="left")
    temp_ts = pd.merge(test[["instance_id","day",feat_1,feat_2]],temp,on=["day",feat_1,feat_2],how="left")
    temp_tr = temp_tr[["instance_id",feat_1+"_"+feat_2 + '_smooth']]
    temp_ts = temp_ts[["instance_id",feat_1+"_"+feat_2 + '_smooth']]
            
            #------填充空值-------------------
    temp_tr = temp_tr.fillna(smooth_cvr_log[feat_1+"_"+feat_2 + '_smooth'])
    temp_ts = temp_ts.fillna(smooth_cvr_log[feat_1+"_"+feat_2 + '_smooth'])
            #----------保存----------
    temp_tr.to_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    temp_ts.to_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    print("保存在 ../data/merge_data/_5_smooth_cvr_train_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    print("保存在 ../data/merge_data/_5_smooth_cvr_test_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    del temp
    gc.collect()
            
for feat_1,feat_2,feat_3 in three_feature:
    savePath = "../data/smooth_cvr/_4_smooth_cvr_{0}_{1}_and_{2}_{3}_{4}.pickle".format(feat_1,feat_2,feat_3,timeWindow[0],timeWindow[1])
    temp = pd.read_pickle(savePath)
    temp_tr = pd.merge(train[["instance_id","day",feat_1,feat_2,feat_3]],temp,on=["day",feat_1,feat_2,feat_3],how="left")
    temp_ts = pd.merge(test[["instance_id","day",feat_1,feat_2,feat_3]],temp,on=["day",feat_1,feat_2,feat_3],how="left")
    temp_tr = temp_tr[["instance_id",feat_1+"_"+feat_2 +"_"+feat_3+'_smooth']]
    temp_ts = temp_ts[["instance_id",feat_1+"_"+feat_2 +"_"+feat_3+'_smooth']]
            
            #------填充空值-------------------
    temp_tr = temp_tr.fillna(smooth_cvr_log[feat_1+"_"+feat_2 +"_"+feat_3+'_smooth'])
    temp_ts = temp_ts.fillna(smooth_cvr_log[feat_1+"_"+feat_2 +"_"+feat_3+'_smooth'])
            #----------保存----------
    temp_tr.to_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    temp_ts.to_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    print("保存在 ../data/merge_data/_5_smooth_cvr_train_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    print("保存在 ../data/merge_data/_5_smooth_cvr_test_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    del temp
    gc.collect()

In [ ]:
#通过中间文件，根据instance_id将有所有的中间表连接起来
smooth_cvr_tr = train_s[["instance_id"]]
smooth_cvr_ts = test_s[["instance_id"]]

In [ ]:
for i in feature_1:
    temp_tr = pd.read_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_merge_0.pickle".format(i))
    temp_ts = pd.read_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_merge_0.pickle".format(i))
    smooth_cvr_tr = pd.merge(smooth_cvr_tr,temp_tr,on=["instance_id"],how="left")
    smooth_cvr_ts = pd.merge(smooth_cvr_ts,temp_ts,on=["instance_id"],how="left")
        
#合并多特征的转化率
for feat_1,feat_2 in combine_feature:
    temp_tr = pd.read_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    temp_ts = pd.read_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_and_{1}_merge_0.pickle".format(feat_1,feat_2))
    smooth_cvr_tr = pd.merge(smooth_cvr_tr,temp_tr,on=["instance_id"],how="left")
    smooth_cvr_ts = pd.merge(smooth_cvr_ts,temp_ts,on=["instance_id"],how="left")
            
for feat_1,feat_2,feat_3 in three_feature:
    temp_tr = pd.read_pickle("../data/merge_data/_5_smooth_cvr_train_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    temp_ts = pd.read_pickle("../data/merge_data/_5_smooth_cvr_test_{0}_{1}_and_{2}_merge_0.pickle".format(feat_1,feat_2,feat_3))
    smooth_cvr_tr = pd.merge(smooth_cvr_tr,temp_tr,on=["instance_id"],how="left")
    smooth_cvr_ts = pd.merge(smooth_cvr_ts,temp_ts,on=["instance_id"],how="left")
            
        
smooth_cvr_tr.to_pickle(save_path_train)
smooth_cvr_ts.to_pickle(save_path_test)
print("保存在 ",save_path_train)
print("保存在 ",save_path_test)

In [ ]:
##########merge data############

In [41]:
train_x = train[(train["day"]>=7)&(train["day"]<=8)][["instance_id","day","is_trade"]]
        
#读取基本的点击行为表
basic_click_tr = pd.read_pickle("../data/merge_data/_5_merge_data_train_basic_click.pickle")
drop_cols = ['user_id', 'context_timestamp','hour', 'item_id', 
                     'item_category_list', 'item_property_list', 'item_brand_id', 
                     'item_city_id','shop_id',
                     'context_page_id']
basic_click_tr = basic_click_tr.drop(drop_cols,axis=1) #删除id类型的字段

In [42]:
basic_click_tr = pd.read_pickle("../data/merge_data/_5_merge_data_train_basic_click.pickle")

In [43]:
train_x = pd.merge(train_x,basic_click_tr,how="left",on=["instance_id"]).drop_duplicates()
del basic_click_tr
gc.collect()
print("合并 basic_click_tr 完毕！！！！")        

合并 basic_click_tr 完毕！！！！


In [44]:
#id类型
id_tr = pd.read_pickle("../data/merge_data/_5_merge_data_train_ID_global_sum.pickle")
train_x = pd.merge(train_x,id_tr,how="left",on=["instance_id"]).drop_duplicates()
del id_tr
gc.collect()
print("合并 id_tr 完毕！！！！")

合并 id_tr 完毕！！！！


In [ ]:
#平滑转化率
smooth_cvr_tr = pd.read_pickle("../data/merge_data/_5_merge_data_train_smooth_cvr_6_8.pickle")
train_x = pd.merge(train_x,smooth_cvr_tr,how="left",on=["instance_id"]).drop_duplicates()
del smooth_cvr_tr
print("合并 smooth_cvr_tr 完毕！！！！")

In [120]:
#原始数据
#origin_tr = pd.read_pickle("../data/merge_data/_5_merge_data_train_originated.pickle")

In [103]:
#train_x = pd.merge(train_x,origin_tr,how="left",on=["instance_id"]).drop_duplicates()
#del origin_tr
#gc.collect()
#print("合并 origin_tr 完毕！！！！")

合并 origin_tr 完毕！！！！


In [47]:
print("拆分训练集和验证集！！！")
#分割训练集和验证集(train and val)
save_path_train = "../data/merge_data/final_train.pickle"
save_path_val = "../data/merge_data/final_val.pickle"
save_path_test = "../data/merge_data/final_test.pickle"
val = train_x[train_x["day_x"]==8]
train = train_x[train_x["day_x"].isin([7,8])]

拆分训练集和验证集！！！


In [48]:
val = val.drop(["day_x","day_y"],axis=1)
train = train.drop(["day_x","day_y"],axis=1)

In [49]:
train = train.drop(["is_trade_y"],axis=1)
val = val.drop(["is_trade_y"],axis=1)

In [50]:
val.to_pickle(save_path_val)
print("验证集已经保存在 {0}".format(save_path_val))        
del val
gc.collect()

验证集已经保存在 ../data/merge_data/final_val.pickle


202

In [51]:
%%time
train.to_pickle(save_path_train)
print("训练集已经保存在 {0}".format(save_path_train))
del train
gc.collect()

训练集已经保存在 ../data/merge_data/final_train.pickle
Wall time: 7min 5s


In [52]:
#############开始合并测试集##################
print("开始合并测试集")
test = test[["instance_id"]]
#点击次数
basic_click_ts = pd.read_pickle("../data/merge_data/_5_merge_data_test_basic_click.pickle")
basic_click_ts = basic_click_ts.drop(drop_cols,axis=1) #删除id类型的字段
test = pd.merge(test,basic_click_ts,how="left",on=["instance_id"]).drop_duplicates()
del basic_click_ts
gc.collect()

开始合并测试集


45

In [53]:
#id类型
id_ts = pd.read_pickle("../data/merge_data/_5_merge_data_test_ID_global_sum.pickle")
test = pd.merge(test,id_ts,how="left",on=["instance_id"]).drop_duplicates()
del id_ts
gc.collect()
        
        
#原始数据
#origin_ts = pd.read_pickle("../data/merge_data/_5_merge_data_test_originated.pickle")
#test = pd.merge(test,origin_ts,how="left",on=["instance_id"]).drop_duplicates()
#del origin_ts
#gc.collect()



21

In [ ]:
#合并平滑转化率文件
smooth_cvr_ts = pd.read_pickle("../data/merge_data/_5_merge_data_test_smooth_cvr_6_8.pickle")
test = pd.merge(test,smooth_cvr_ts,how="left",on=["instance_id"]).drop_duplicates()
del smooth_cvr_ts

In [55]:
test.to_pickle(save_path_test)
print("测试集保存在 {0}".format(save_path_test))

测试集保存在 ../data/merge_data/final_test.pickle
